In [29]:
import pandas as pd
import numpy as np
import itertools
import shutil

### Read Data

In [1]:
data_path = "/path/to/data/"

In [3]:
data = pd.read_csv(data_path + "train.data.v1.txt", delimiter="\t", header=None, names=["keyword", "context", "img_1", "img_2", "img_3", "img_4", "img_5", "img_6", "img_7", "img_8", "img_9", "img_10"])
keys = pd.read_csv(data_path+"train.gold.v1.txt", delimiter="\t", header=None)
data["gold_key"] = keys
data

,keyword,context,img_1,img_2,img_3,img_4,img_5,img_6,img_7,img_8,img_9,img_10,gold_key
0,moorhen,moorhen swamphen,image.3.jpg,image.8.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.0.jpg,image.5.jpg,image.6.jpg,image.7.jpg,image.9.jpg,image.0.jpg
1,serinus,serinus genus,image.3.jpg,image.23.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.20.jpg,image.5.jpg,image.24.jpg,image.22.jpg,image.21.jpg,image.20.jpg
2,pegmatite,pegmatite igneous,image.41.jpg,image.39.jpg,image.42.jpg,image.43.jpg,image.40.jpg,image.44.jpg,image.37.jpg,image.38.jpg,image.36.jpg,image.35.jpg,image.35.jpg
3,bangalores,bangalores torpedo,image.58.jpg,image.59.jpg,image.64.jpg,image.57.jpg,image.55.jpg,image.56.jpg,image.62.jpg,image.63.jpg,image.61.jpg,image.60.jpg,image.55.jpg
4,bonxie,bonxie skua,image.3.jpg,image.77.jpg,image.78.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.79.jpg,image.76.jpg,image.75.jpg,image.75.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12864,bomarea,bomarea genus,image.11820.jpg,image.3.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.1559.jpg,image.1285.jpg,image.5.jpg,image.6482.jpg,image.10937.jpg,image.11820.jpg
12865,tragopogon,tragopogon genus,image.3.jpg,image.6250.jpg,image.15001.jpg,image.4.jpg,image.1.jpg,image.2.jpg,image.12074.jpg,image.5.jpg,image.4087.jpg,image.12806.jpg,image.12074.jpg
12866,illustrator,illustrator artist,image.10633.jpg,image.723.jpg,image.13372.jpg,image.881.jpg,image.12635.jpg,image.726.jpg,image.5985.jpg,image.722.jpg,image.724.jpg,image.725.jpg,image.10633.jpg
12867,brunfelsia,brunfelsia genus,image.3.jpg,image.8911.jpg,image.5195.jpg,image.4.jpg,image.12827.jpg,image.1.jpg,image.2.jpg,image.5.jpg,image.9129.jpg,image.8605.jpg,image.8911.jpg


### Split the data into 5 subsets

In [4]:
data_shuffled = data.sample(frac=1)
data_split = np.array_split(data_shuffled, 5)
assigned_names = ["LeYuan", "Insaf", "Aryss", "Poon", "Aung"]

for df, name in zip(data_split, assigned_names):
    df.to_csv("data_split_for_"+name+".csv", index=False)

### Get unique set of images

In [24]:
subset_data = pd.read_csv("data_split_for_Aung.csv")       # change the file name
image_list = list(set(list(itertools.chain.from_iterable(df[[f"img_{i}" for i in range(1, 11)]].values.tolist()))))
print("length: ", len(image_list))

length:  7139


### Copy the images from the list to new folder

In [30]:
origin_path = "/path/to/images/location/"
target_path = "/new/folder/location/"

for file in image_list:
    shutil.copyfile(origin_path + file, target_path + file)

print(f"Copy Successfully: {len(image_list) == len(glob.glob(target_path+'*'))}")